In [1]:
import pandas as pd

import LotData.LotData as ld
import LotData.ExtractorsAndTables as ent
import utility.LoggingUtility as lut
import utility.webscrapingUtil as wut
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,table
import CW_Scraper
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine.reflection import Inspector
import time
from decouple import Config, RepositoryEnv
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from database.EnvSettings import environment_information
from psycopg2 import connect
from sqlalchemy import select, and_, Inspector, inspect
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import Table, MetaData
import numpy as np
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from Runnables.RunningSettings import wanted_categories
import database.DatabaseManager as dbm
import datetime
import pytz

In [2]:
class DatabaseManager:

    def __init__(self,session,engine):
        self.session = session
        self.engine = engine

    def getAllTableNames(self):
        query = f"SELECT table_name FROM information_schema.tables WHERE table_schema='public' AND table_type='BASE TABLE';"
        result = self.session.execute(text(query))
        all_table_names = [name_tuple[0] for name_tuple in result.fetchall()]
        return  all_table_names # or fetch the result as required


    """

    @return true if LID is found in table, false if not - fails if the table doesn't have a lid column.
    """
    def exists(self,LID,table):
        query = f"SELECT EXISTS (SELECT 1 FROM {table} WHERE lid = :lid)"

        result = self.session.execute(text(query), {'lid': LID})
        return result.scalar()

    """

    Checks whether or not all tables have the specified LID - returns the tables that do not have that lid

    """

    def tablesWithout(self,LID):
        all_tables = self.getAllTableNames()
        tables_without = [table_name for table_name in all_tables if not self.exists(LID,table_name)]
        return tables_without

    def insert(self,table,dataframe):
        try:
            dataframe.to_sql(table, con=self.engine, if_exists='append', index=False)
            return (dataframe,None)
        except Exception as e:
            return (None,e)

    def update(self,table,lid,column,value):
        try:
            return (self.session.execute(f"UPDATE {table} SET {column} = {value} WHERE {table}.lid = {lid}"),None)
        except Exception as e:
            return (False,e)

    def hasFinalBid(self,LID):
        query = f"SELECT EXISTS (SELECT 1 FROM bid WHERE lid = :lid AND is_final_bid = True)"

        result = self.session.execute(text(query), {'lid': LID})
        return result.scalar()

    def isClosed(self,LID):
        query = f"SELECT EXISTS (SELECT 1 FROM auction_history WHERE lid = :lid AND is_closed = True)"

        result = self.session.execute(text(query), {'lid': LID})
        return result.scalar()

    def getBiddingCloseTimestamp(self,LID):
        query = text("""
        SELECT MAX(bidding_close_timestamp) AS latest_bidding_close_timestamp
        FROM (
            SELECT bidding_close_timestamp
            FROM auction_history
            WHERE lid = :lid_value

            UNION ALL

            SELECT bidding_close_timestamp
            FROM auction
            WHERE lid = :lid_value
        ) AS combined_results;
        """)

        result = self.session.execute(query, {'lid_value': LID}).scalar()
        return result


    def validateRecordDataframe(self, DF, table_name,unique_constraints):

        # Reflect the table
        metadata = MetaData()
        table = Table(table_name, metadata, autoload_with=self.engine)

        # Initialize an empty list to hold rows to insert
        rows_to_insert = []

        # Iterate over each row in the DataFrame
        for index, row in DF.iterrows():
            # Build the filter condition for the unique constraints
            filter_condition = and_(*[table.c[col] == row[col] for col in unique_constraints])

            # Check if the row already exists
            exists_query = select(table).where(filter_condition)
            result = self.session.execute(exists_query).fetchone()

            # If the row does not exist, add it to the list to insert
            if result is None:
                rows_to_insert.append(row)

        # Convert the rows to insert back to a DataFrame
        rows_to_insert_df = pd.DataFrame(rows_to_insert)

        return rows_to_insert_df

    def getRecordConstraints(self,table_name):
        insepctor = inspect(self.engine)

        if table_name in ["meta","bid","spec","auction"]:
            return insepctor.get_pk_constraint(table_name)[0]["column_names"]

        if table_name in ["shipping","image"]:
            return insepctor.get_unique_constraints(table_name)[0]["column_names"]

        if table_name in ["auction_history","favorite_history"]:
            return []

    def insertRecordDataframe(self,DF,table_name):
        unique_constraints = self.getRecordConstraints(table_name)
        valid_dataframe = self.validateRecordDataframe(DF,table_name,unique_constraints)

        result = self.insert(table_name,valid_dataframe)
        #TODO: Add a check for bid

        return result

In [3]:
session,engine = dbm.getSessionEngine()
LID_to_test = 84682673


meta_data = ent.MetadataExtractor(LID_to_test,0,333,"watches")
ld_lotdata = ld.LotData(meta_data)
db_manager = dbm.DatabaseManager(session,engine)
bid_record_df = ld_lotdata["bid_record"]
ship_record_df = ld_lotdata["shipping_record"]
insepctor = inspect(engine)

In [4]:
db_manager.insertRecordDataframe(bid_record_df,"bid")

[      bid_id  amount currency_code  from_order explanation_type  \
 0  411348861   145.0           EUR       False             None   
 
    bid_placed_timestamp  bidder_name  \
 0  2024-06-19T18:29:46Z  Bidder 5860   
 
                                bidder_token bidder_country_code  \
 0  822311e208c3a0af9ee228c86deb31638f264953                  be   
 
                  bid_api_timestamp  is_final_bid  is_reserve_price_met  \
 0 2024-07-01 21:23:08.240048+00:00          True                    -1   
 
    favorite_count     aid       lid             latest_bid_timestamp  
 0              27  958977  84682673 2024-07-01 21:23:08.001394+00:00  ,
 None]

In [5]:
insepctor.get_pk_constraint("meta")

{'constrained_columns': ['lid'], 'name': 'meta_pkey', 'comment': None}